In [10]:
from settrade_v2 import Investor
import pandas as pd
import datetime
from cassandra.cluster import Cluster
import time

In [11]:
# 1. เชื่อมต่อ Database
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()


session.set_keyspace('stock_data')

In [12]:
# 4. Connect Settrade
investor = Investor(
    app_id="eX5XjBgNX6OoXZ14",                                 
    app_secret="ES2EcRZes9SWxnry+8jt6m3Ic+wpD/RiT8bDDp6jjvA=", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)   
market = investor.MarketData() # เปลี่ยนจาก .Market() เป็น .MarketData()
print("เชื่อมต่อ Settrade API และเข้าถึง Market Data สำเร็จ!")

# 5. ทดลองดึงข้อมูล
symbol = "PTT"
candlestick = market.get_candlestick(
    symbol=symbol, 
    interval="1d", 
    limit=10
)
print(candlestick)



เชื่อมต่อ Settrade API และเข้าถึง Market Data สำเร็จ!
{'lastSequence': 440815, 'time': [1766422800, 1766509200, 1766595600, 1766682000, 1766941200, 1767027600, 1767546000, 1767632400, 1767718800, 1767805200], 'open': [32.0, 31.75, 31.75, 31.5, 31.5, 31.5, 31.75, 31.75, 31.5, 33.25], 'high': [32.25, 32.0, 31.75, 31.75, 31.75, 32.0, 32.0, 31.75, 31.75, 36.25], 'low': [31.75, 31.5, 31.5, 31.25, 31.25, 31.5, 31.5, 31.5, 31.5, 31.0], 'close': [31.75, 31.75, 31.5, 31.5, 31.5, 32.0, 31.75, 31.75, 31.5, 31.5], 'volume': [55660625, 30927077, 20536806, 34084620, 38891304, 41210512, 35819490, 41857471, 21255014, 708200], 'value': [0, 0, 0, 0, 0, 0, 0, 0, 0, 22748000.0]}


In [13]:
print(dir(investor))

['Derivatives', 'Equity', 'MarketData', 'RealtimeDataConnection', 'SANDBOX', 'SANDBOX_BROKER_ID', 'SANDBOX_ENV', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_app_code', '_app_id', '_app_secret', '_broker_id', '_ctx', '_is_auto_queue']


In [14]:
import time
from datetime import datetime
import pandas as pd

# 1. ฟังก์ชันดึง candlestick
def get_candlestick_data(symbol):
    try:
        symbol = symbol.strip().upper()
        res = market.get_candlestick(
            symbol=symbol,
            interval="1d",
            limit=1000
        )
        return res
    except Exception as e:
        print(f"❌ API Error {symbol}: {e}")
        return None

# 2. ฟังก์ชัน insert ข้อมูล (เปลี่ยน ticker -> symbol ใน SQL)
def insert_historical_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ {symbol}: ไม่มีข้อมูลในระบบ Sandbox")
        return

    time_list = res['time']
    num_rows = len(time_list)
    
    for i in range(num_rows):
        try:
            timestamp = datetime.fromtimestamp(time_list[i])
            
            # แก้ไขจาก ticker เป็น symbol ตามชื่อคอลัมน์ใหม่
            session.execute("""
                INSERT INTO candlestick_data (
                    symbol, date, open, high, low, close, volume
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                symbol,                    
                timestamp,                 
                float(res['open'][i]),     
                float(res['high'][i]),     
                float(res['low'][i]),      
                float(res['close'][i]),    
                int(res['volume'][i])      
            ))
        except Exception as e:
            print(f"❌ {symbol} บรรทัดที่ {i} ผิดพลาด: {e}")
            return 

    print(f"✅ บันทึก {symbol} สำเร็จ {num_rows} แถว")

# 3. โหลดรายชื่อหุ้นและเริ่มรัน (Limit 100)
try:
    df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/stock_pattern/data/settrade1.xlsx")
    stock_list = df['หลักทรัพย์'].dropna().unique().tolist()
    if 'หลักทรัพย์' in stock_list:
        stock_list.remove('หลักทรัพย์')
    
    # เอาแค่ 100 ตัวแรก
    stock_list = stock_list[:] 
    
    print(f"🔎 พบรายชื่อหุ้น {len(stock_list)} ")

    for symbol in stock_list:
        symbol = symbol.strip().upper()
        print(f"🚀 กำลังเริ่มดึง: {symbol}...", end=" ")
        
        res = get_candlestick_data(symbol)
        insert_historical_data(symbol, res)
        
        time.sleep(1) 

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")

🔎 พบรายชื่อหุ้น 923 
🚀 กำลังเริ่มดึง: 24CS... ❌ API Error 24CS: Symbol not found [24CS]
⚠️ 24CS: ไม่มีข้อมูลในระบบ Sandbox
🚀 กำลังเริ่มดึง: 2S... ❌ 2S บรรทัดที่ 0 ผิดพลาด: Error from server: code=2200 [Invalid query] message="Undefined column name date in table stock_data.candlestick_data"
🚀 กำลังเริ่มดึง: 3BBIF... ❌ 3BBIF บรรทัดที่ 0 ผิดพลาด: Error from server: code=2200 [Invalid query] message="Undefined column name date in table stock_data.candlestick_data"
🚀 กำลังเริ่มดึง: A... ❌ A บรรทัดที่ 0 ผิดพลาด: Error from server: code=2200 [Invalid query] message="Undefined column name date in table stock_data.candlestick_data"


KeyboardInterrupt: 

In [10]:
# ต้องใส่ ALLOW FILTERING เพราะเราไม่ได้ระบุช่วงเวลา (Clustering Key) ครบถ้วนในบางกรณี
query = "SELECT * FROM candlestick_data WHERE symbol = 'ADVANC' LIMIT 10 ALLOW FILTERING;"
rows = session.execute(query)
df_3bb = pd.DataFrame(list(rows))

print("ข้อมูลหุ้น ADVANC:")
print(df_3bb)

ข้อมูลหุ้น ADVANC:
   symbol       date  close   high    low   open    volume
0  ADVANC 2022-12-27  193.5  194.0  192.0  192.5   5128851
1  ADVANC 2022-12-28  194.0  195.0  193.5  194.0   3692182
2  ADVANC 2022-12-29  196.0  196.0  193.5  194.0   4648328
3  ADVANC 2022-12-30  195.0  197.0  194.5  196.0   6036469
4  ADVANC 2023-01-03  194.5  196.5  194.5  196.0   2993519
5  ADVANC 2023-01-04  197.0  197.5  194.0  194.5   5079778
6  ADVANC 2023-01-05  201.0  202.0  196.5  197.0  13629702
7  ADVANC 2023-01-06  201.0  202.0  199.5  201.0   4263966
8  ADVANC 2023-01-09  203.0  203.0  201.0  202.0   4564272
9  ADVANC 2023-01-10  203.0  204.0  202.0  204.0   4854592
